In [187]:
%pip install beautifulsoup4

116372.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


We're going to extract the following from each letter: 

- the topic the writer is talking about 
- the text itself
- the author's name and location

we'll use BeautifulSoup to do this. 

why not to use regular expressions: 

https://stackoverflow.com/questions/1732348/regex-match-open-tags-except-xhtml-self-contained-tags

In [188]:
with open('/workspaces/long-island-digest/data/raw/newsday/letters/accessory-dwelling-unit-affordable-housing-racism-s68193.html', 'r') as fin:
    html_doc = fin.read()


In [189]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc)

We can try getting all the text at once, but we can see the problem there is a lot of text on the page that we're not interested in. 

In [190]:
soup.getText()

'Illegal housing units need attention - NewsdaySkip to contentMenuToday\'s paperLog inSubscribe $1 FOR 5 MONTHSLong IslandSportsNewsday TVNewsFeedMeOpinionBusinessObituariesNewsday LiveSearchOpinionLettersIllegal housing units need attentionHempstead Town Supervisor Don Clavin, right, Oyster Bay Town Supervisor Joseph Saladino, left,\xa0and others call on Gov. Kathy Hochul to remove the accessory dwelling unit provision in her budget. Credit: Howard SchnappBy Newsday ReadersFebruary 27, 2022   Illegal housing units need attentionMany families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such ["Hochul pulls housing plan," News, Feb. 18].If everyone is so against making them legal, then why haven\'t our local governments enforced the laws? They turn a blind eye to multiple meters on houses and garages that house all these illegal apartments

We can actually get a lot of what we want by looking at the h3 elements. 

In [191]:
soup.findAll('h3')

[<h3>Illegal housing units need attention</h3>,
 <h3>Russia has forfeited Security Council seat</h3>,
 <h3>We must learn how to develop a conscience</h3>,
 <h3>Quick Links</h3>,
 <h3>Partners</h3>,
 <h3>News</h3>,
 <h3>More</h3>]

At least in the one page that we're looking at, there's a structure to letters: 

Subject Heading
    Letter Contents
    Author


this maps to 
h3
 p
 p <strong>

 there can bee more than one letter for the same subject; guessing we'll have some other surprises as we go along, too. 

In [192]:
letter_content = soup.find(name="div", attrs={"class": "contentAccess"})

We now perform control break processing on the data. The H3 elements form topics, the paragraph tags below them contain text, and the strong tags contain the author's name and hometown.

In [193]:
articles = []
topic = ""
text = "" 
author = ""

for x in letter_content.find_all():
    # if h3, reset the topic
    if x.name == "h3":
        topic = x.get_text()

    # this is a paragraph of text
    if x.name == "p":
        if x.find("strong"):
            if x.find("strong").text == x.text:
                author = x.text
                print(topic, text, author)
                articles.append( {
                    "topic": topic,
                    "author": author,
                    "text": text
                })
                text = ""
                author = ""
        else:
            text = text + x.get_text() + '\n' 

Illegal housing units need attention Many families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such ["Hochul pulls housing plan," News, Feb. 18].
If everyone is so against making them legal, then why haven't our local governments enforced the laws? They turn a blind eye to multiple meters on houses and garages that house all these illegal apartments. How about landlords who rent out mattress space? This is missed revenue that puts a burden on homeowners, schools, fire departments, police, etc.
Why not make these landlords pay additional tax and take responsibility for their property? How about the absentee landlords who reap the rent money and don’t care about the neighborhood?
Those of us who live in towns have no defense and are ignored by local governments about upholding the law on these illegal dwellings.
 Barbara Keilty Michaleski,

The results are extremely promising!

In [194]:
import pandas as pd
pd.DataFrame(articles)

,topic,author,text
0,Illegal housing units need attention,"Barbara Keilty Michaleski, West Hempstead","Many families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such [""Hochul pulls housing plan,"" News, Fe..."
1,Illegal housing units need attention,"Ray Roel, East Northport","Long Island’s affordable housing ""needs"" run counter to its need for clean water [""Housing plan worth retooling,"" Editorial, Feb. 22]. Our sole source of water, our aquifers, cannot sustain further development, period. We must learn how to live with no..."
2,Illegal housing units need attention,"Bob Rotunda, Islip","It is fascinating to read the articles, editorials and pros and cons regarding Gov. Kathy Hochul's rescinded accessory housing proposal. It almost seems as if the idea of basement apartments, garage conversions, exterior staircases, and mother-daughter..."
3,Illegal housing units need attention,"Don Otlin, Franklin Square","My wife and I moved to Nassau County from Queens 46 years ago to get away from the ""hustle and bustle"" of congested city life and crowded places. But we have seen both Nassau and Suffolk counties become too crowded with more people, more traffic, more ..."
4,Illegal housing units need attention,"Elizabeth Leyser, Hicksville","The following comments appeared in an email I had received from the Town of Oyster Bay: ""Dozens of angry and outraged officials at the state, county, town, city and village levels joined to denounce the state’s plan to eliminate single family home zoni..."
5,Russia has forfeited Security Council seat,"Kurt Miller, Plainview","Russia has grossly violated the United Nations' main objectives of being the keeper of international peace and security, the promotion of the well-being of the world's people , and international cooperation to these ends [""Putin: Military operation set..."
6,Russia has forfeited Security Council seat,"Martin Geller, Manhasset","The overarching reason that Russian President Vladimir Putin has declared war on the Ukraine is this — his pride. To Putin, the collapse of the Soviet Union was a monumental tragedy, and its demise was a dagger in Putin’s heart. Ever since Ukraine rega..."
7,Russia has forfeited Security Council seat,"Martin Blumberg, Melville","There is no justification for Russia’s invasion of Ukraine. A new harsher wave of sanctions will not stop Russian aggression, and they will not stop until they take over the government. This is an atrocity to world peace and a sad day for the whole wor..."
8,We must learn how to develop a conscience,"Jules Jacobs, East Islip","I'm concerned with parents all over this country trying to protect their children from learning about the horrors that people have committed against each other [""Opening windows on definition of racism,"" Opinion, Feb. 10]. I'm concerned because what th..."


In [195]:
from bs4 import BeautifulSoup
import pandas as pd

def process_letters_to_editor(html_file):

    with open(html_file) as fin:
        html_doc = fin.read()

    soup = BeautifulSoup(html_doc)
    letter_content = soup.find(name='div', attrs={"class": "contentAccess"})

    articles = []
    topic = ""
    text = "" 
    author = ""

    contents = letter_content.find_all(["h1", "h2", "h3", "p"])
    for x in contents:
        # if h3, reset the topic
        if x.name in ["h1", "h2", "h3"]:
            topic = x.get_text()

        # this is a paragraph of text
        if x.name == "p":
            if x.find("strong"):
                if x.find("strong").text == x.text:
                    author = x.text
                    articles.append( {
                        "topic": topic,
                        "author": author,
                        "text": text
                    })
                    text = ""
                    author = ""
            else:
                text = text + x.get_text() + '\n' 

    return pd.DataFrame(articles)

In [196]:
Y = process_letters_to_editor('/workspaces/long-island-digest/data/raw/newsday/letters/accessory-dwelling-unit-kathy-hochul-j37633.html')

We can see a funny result here - the honorific for Bruce Sander has been attached to the next story. Not going to deal with it just yet. 

In [197]:
Y

,topic,author,text
0,,"— Bruce Sander, Stony Brook","The Stony Brook Concerned Homeowners has been battling to preserve single-family neighborhoods against illegal college boarding houses for more than eight years [""Hochul’s stumble on housing needs,"" Editorial, Feb. 9]. If not for Brookhaven Town’s juri..."
1,,"— Stuart Abbott, North Bellmore","The writer is founder and president of the Stony Brook Concerned Homeowners. \n \nMy parents moved to Long Island in 1959 to escape overcrowding in the Bronx. Lured by wide-open spaces, they moved into a modest house in a neighborhood with good school..."
2,,"— Cathy Greene, East Setauket","Before ""Granny"" or any other homeowner thinks it’s a windfall becoming a landlord, they had better be an expert in New York State landlord-tenant law. Any well-seeming tenant can stop paying rent. The landlord must give them up to 90 days’ notice to va..."
3,,"— Susan B. Lyons, Freeport","The writer has been a Long Island landlord since 1990.\n \nI oppose Gov. Kathy Hochul’s plan. It is an ill-conceived stopgap that not only does not come near to solving any housing and economic problems New Yorkers face, but also would be detrimental ..."
4,,"— Gloria R. Maffettone, Uniondale","\nWill Gov. Kathy Hochul’s private home be in an area that will be inundated with ""accessory dwelling units""? Will these now-illegal living spaces become legal with little thought given to proper living conditions, such as how many rooms will be perm..."
5,,"— Marc de Venoge, Manhasset","The writer is a retired school administrator.\n \nI am sickened by Gov. Kathy Hochul’s ""affordable housing plan,"" which effectively eliminates zoning and would no doubt ramp up density, congestion and real estate taxes on Long Island. Taking zoning de..."
6,,"— Richard M. Frauenglass, Huntington","\nGov. Kathy Hochul’s hokum is her proposed solution for ""affordable housing."" To suggest that accessory apartments can be built at owner discretion is inviting the slow deterioration of neighborhoods.\nSuburban residential communities are for single..."
7,,"— John Minogue, Manhasset","\nIt is hoped that Gov. Kathy Hochul’s plan includes a provision stating that the basement, attic, garage and any bedroom in the governor’s mansion that is not being used by an immediate family member shall be converted to accessory dwelling units. \n"


In [198]:
X = process_letters_to_editor('/workspaces/long-island-digest/data/raw/newsday/letters/aggressive-driving-marijuana-licenses-housing-development-mlb-a59429.html')

In [199]:
from glob import glob

html_files = glob("/workspaces/long-island-digest/data/raw/newsday/letters/*.html")
html_files[:5]

['/workspaces/long-island-digest/data/raw/newsday/letters/1-5-million-is-not-frivolous-to-suffolk-county-taxpayers-q32984.html',
 '/workspaces/long-island-digest/data/raw/newsday/letters/a-bipartisan-idea-for-tainted-donors-s42502.html',
 '/workspaces/long-island-digest/data/raw/newsday/letters/a-cup-of-coffee-is-just-a-courtesy-d89501.html',
 '/workspaces/long-island-digest/data/raw/newsday/letters/a-new-arena-no-fix-the-roads-yes-t33374.html',
 '/workspaces/long-island-digest/data/raw/newsday/letters/a-solution-for-nassau-s-77-acres-m42091.html']

In [200]:
articles = []
for h in html_files[:10]:
    a = process_letters_to_editor(h)
    articles.append(a)

Looking at 10 articles shows that we can't really ignore the problem. Let's improve on our processor in these ways: 
- Add ability to get a main topic if it's outside of the container. 
- Remove honorific blurbs (for now)

In [201]:
articles_df = pd.concat(articles).reset_index(drop=True)
articles_df

,topic,author,text
0,Illegal housing units need attention,"Barbara Keilty Michaleski, West Hempstead","Many families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such [""Hochul pulls housing plan,"" News, Fe..."
1,Illegal housing units need attention,"Ray Roel, East Northport","Long Island’s affordable housing ""needs"" run counter to its need for clean water [""Housing plan worth retooling,"" Editorial, Feb. 22]. Our sole source of water, our aquifers, cannot sustain further development, period. We must learn how to live with no..."
2,Illegal housing units need attention,"Bob Rotunda, Islip","It is fascinating to read the articles, editorials and pros and cons regarding Gov. Kathy Hochul's rescinded accessory housing proposal. It almost seems as if the idea of basement apartments, garage conversions, exterior staircases, and mother-daughter..."
3,Illegal housing units need attention,"Don Otlin, Franklin Square","My wife and I moved to Nassau County from Queens 46 years ago to get away from the ""hustle and bustle"" of congested city life and crowded places. But we have seen both Nassau and Suffolk counties become too crowded with more people, more traffic, more ..."
4,Illegal housing units need attention,"Elizabeth Leyser, Hicksville","The following comments appeared in an email I had received from the Town of Oyster Bay: ""Dozens of angry and outraged officials at the state, county, town, city and village levels joined to denounce the state’s plan to eliminate single family home zoni..."
5,Russia has forfeited Security Council seat,"Kurt Miller, Plainview","Russia has grossly violated the United Nations' main objectives of being the keeper of international peace and security, the promotion of the well-being of the world's people , and international cooperation to these ends [""Putin: Military operation set..."
6,Russia has forfeited Security Council seat,"Martin Geller, Manhasset","The overarching reason that Russian President Vladimir Putin has declared war on the Ukraine is this — his pride. To Putin, the collapse of the Soviet Union was a monumental tragedy, and its demise was a dagger in Putin’s heart. Ever since Ukraine rega..."
7,Russia has forfeited Security Council seat,"Martin Blumberg, Melville","There is no justification for Russia’s invasion of Ukraine. A new harsher wave of sanctions will not stop Russian aggression, and they will not stop until they take over the government. This is an atrocity to world peace and a sad day for the whole wor..."
8,We must learn how to develop a conscience,"Jules Jacobs, East Islip","I'm concerned with parents all over this country trying to protect their children from learning about the horrors that people have committed against each other [""Opening windows on definition of racism,"" Opinion, Feb. 10]. I'm concerned because what th..."
9,,"— Bruce Sander, Stony Brook","The Stony Brook Concerned Homeowners has been battling to preserve single-family neighborhoods against illegal college boarding houses for more than eight years [""Hochul’s stumble on housing needs,"" Editorial, Feb. 9]. If not for Brookhaven Town’s juri..."


In [202]:
articles_df.iloc[13]['text']

'\u2003 \nWill Gov. Kathy Hochul’s private home be in an area that will be inundated with "accessory dwelling units"? Will these now-illegal living spaces become legal with little thought given to proper living conditions, such as how many rooms will be permitted in a garage, or will basement areas need egresses added? And how many "backyard cottages" are permitted on what size properties? \nWhat about the impact on local schools? More children to be educated. Did Hochul discuss this with school administrators before issuing her plan? There will be increases in garbage and cars parked on local streets. All of this needs to be addressed before this plan is put into effect. \n'

In [203]:
import json

def process_letters_to_editor_improved(html_file):

    with open(html_file) as fin:
        html_doc = fin.read()

    soup = BeautifulSoup(html_doc)
    letter_content = soup.find(name='div', attrs={"class": "contentAccess"})

    articles = []
    topic = soup.main.article.header.h1.text
    metadata = soup.find(name="script", attrs={"type": "application/ld+json"})
    metadata_json = json.loads(metadata.get_text())
    published_date = metadata_json['datePublished']
    text = "" 
    author = ""

    # look for a main topic outside of the content div

    contents = letter_content.find_all(["h1", "h2", "h3", "p"])
    for x in contents:
        # if h3, reset the topic
        if x.name in ["h1", "h2", "h3"]:
            topic = x.get_text()

        # this is a paragraph of text
        if x.name == "p":
            if x.find("strong"):
                if x.find("strong").text == x.text:
                    author = x.text
                    articles.append( {
                        "topic": topic,
                        "author": author,
                        "text": text,
                        "date_published": published_date,
                    })
                    text = ""
                    author = ""
            else:
                if not x.find("em"):
                    text = text + x.get_text() + '\n' 

    return pd.DataFrame(articles)

This looks a little better. We have a topic for each letter and the honorifics have been removed. Also added the publishing date for the letters. 

In [204]:
articles = []
for h in html_files[:10]:
    a = process_letters_to_editor_improved(h)
    articles.append(a)

articles_df = pd.concat(articles).reset_index(drop=True)
articles_df

,topic,author,text,date_published
0,Illegal housing units need attention,"Barbara Keilty Michaleski, West Hempstead","Many families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such [""Hochul pulls housing plan,"" News, Fe...",2022-02-27T10:00:00Z
1,Illegal housing units need attention,"Ray Roel, East Northport","Long Island’s affordable housing ""needs"" run counter to its need for clean water [""Housing plan worth retooling,"" Editorial, Feb. 22]. Our sole source of water, our aquifers, cannot sustain further development, period. We must learn how to live with no...",2022-02-27T10:00:00Z
2,Illegal housing units need attention,"Bob Rotunda, Islip","It is fascinating to read the articles, editorials and pros and cons regarding Gov. Kathy Hochul's rescinded accessory housing proposal. It almost seems as if the idea of basement apartments, garage conversions, exterior staircases, and mother-daughter...",2022-02-27T10:00:00Z
3,Illegal housing units need attention,"Don Otlin, Franklin Square","My wife and I moved to Nassau County from Queens 46 years ago to get away from the ""hustle and bustle"" of congested city life and crowded places. But we have seen both Nassau and Suffolk counties become too crowded with more people, more traffic, more ...",2022-02-27T10:00:00Z
4,Illegal housing units need attention,"Elizabeth Leyser, Hicksville","The following comments appeared in an email I had received from the Town of Oyster Bay: ""Dozens of angry and outraged officials at the state, county, town, city and village levels joined to denounce the state’s plan to eliminate single family home zoni...",2022-02-27T10:00:00Z
5,Russia has forfeited Security Council seat,"Kurt Miller, Plainview","Russia has grossly violated the United Nations' main objectives of being the keeper of international peace and security, the promotion of the well-being of the world's people , and international cooperation to these ends [""Putin: Military operation set...",2022-02-27T10:00:00Z
6,Russia has forfeited Security Council seat,"Martin Geller, Manhasset","The overarching reason that Russian President Vladimir Putin has declared war on the Ukraine is this — his pride. To Putin, the collapse of the Soviet Union was a monumental tragedy, and its demise was a dagger in Putin’s heart. Ever since Ukraine rega...",2022-02-27T10:00:00Z
7,Russia has forfeited Security Council seat,"Martin Blumberg, Melville","There is no justification for Russia’s invasion of Ukraine. A new harsher wave of sanctions will not stop Russian aggression, and they will not stop until they take over the government. This is an atrocity to world peace and a sad day for the whole wor...",2022-02-27T10:00:00Z
8,We must learn how to develop a conscience,"Jules Jacobs, East Islip","I'm concerned with parents all over this country trying to protect their children from learning about the horrors that people have committed against each other [""Opening windows on definition of racism,"" Opinion, Feb. 10]. I'm concerned because what th...",2022-02-27T10:00:00Z
9,Hochul's accessory apartment plan is hokum,"— Bruce Sander, Stony Brook","The Stony Brook Concerned Homeowners has been battling to preserve single-family neighborhoods against illegal college boarding houses for more than eight years [""Hochul’s stumble on housing needs,"" Editorial, Feb. 9]. If not for Brookhaven Town’s juri...",2022-02-13T10:00:00Z


We can scale out now. 

In [205]:
articles = []
for h in html_files:
    a = process_letters_to_editor_improved(h)
    articles.append(a)
articles_df


KeyboardInterrupt: 

In [ ]:
articles_df = pd.concat(articles).reset_index(drop=True)
articles_df.topic.value_counts()

Uproar about mask mandate                              13
Russia's Ukraine invasion: Addressing the U.S. role    12
Taking sides on Blakeman                               11
Different takes on the Ukraine invasion                10
Ukraine dominates global outlook                       10
                                                       ..
Canceling baseball a big disappointment                 1
Keep dangerous drivers off the roads                    1
Many students won’t pass this test                      1
Other ways to deal with debt collectors                 1
It takes a long time to see a doctor                    1
Name: topic, Length: 235, dtype: int64

We can see that there are some people who write multiple notes. 

In [ ]:
articles_df.author.value_counts()

— Carol Ludwig, Wantagh                    2
— Glenn Tyranski, Huntington               2
— Jim Kiernan, Holbrook                    2
— Alexa Marinos, North Babylon             2
— Christopher L. Turpin, Patchogue         2
                                          ..
— Jay Fried, Kings Park                    1
— Leroy Bombardiere, Huntington Station    1
— Richard Siegelman, Plainview             1
— Gary Anderson, Smithtown                 1
Edward Hughes, Floral Park                 1
Name: author, Length: 408, dtype: int64

In [ ]:
articles_df[articles_df.author.str.contains('Ludwig')].text.values

array(['Officers in Nassau and Suffolk counties’ police departments, who were hired to protect us, shielded a colleague regarding the arrest of an innocent man ["Shot cabbie wins $2M settlement," News, March 8]. The punishment was that only this officer was fired, and Nassau (taxpayers) must fork over $2 million to the victim, with more to come from Suffolk. \nTo spare innocent taxpayers — and as a deterrent for unprofessional behavior — why is it not mandatory that all public servants carry personal liability insurance? \n',
       'A photo on the top of a page shows the face of a Wantagh High School wrestling match winner atop his opponent ["Wantagh anchors win it," High School Sports, Jan. 23]. The face of his opponent is not shown. \nAnother photo, on the bottom of the page, also shows the winner on top of his opponent ["CSH fends off unexpected foe"]. The face of this losing opponent, though, is shown. \nWith teenage years being so important in the development of self-image, I app

In [ ]:
articles_df['date_published_utc'] = articles_df['date_published'].apply(lambda x: pd.to_datetime(x))
articles_df.set_index('date_published_utc').resample('M').count()

,topic,author,text,date_published
date_published_utc,,,,
2022-01-31 00:00:00+00:00,154,154,154,154
2022-02-28 00:00:00+00:00,145,145,145,145
2022-03-31 00:00:00+00:00,114,114,114,114


In [ ]:
articles_df.to_parquet('/workspaces/long-island-digest/data/processed/letters-2022-only.parquet')

In [ ]:
with open('/workspaces/long-island-digest/data/raw/newsday/letters/accessory-dwelling-unit-kathy-hochul-j37633.html', 'r') as fin:
    html_doc = fin.read()

soup = BeautifulSoup(html_doc)

In [ ]:
lc = soup.find("div", attrs={"class": "contentAccess"})

In [ ]:
soup.main.article.header.h1.text

"Hochul's accessory apartment plan is hokum"

In [ ]:
p = lc.find(name="strong").parent

In [ ]:
p.next_sibling.child

<p><em>The writer is founder and president of the Stony Brook Concerned Homeowners.</em> </p>

there's a lot of interesting stuff buried in the headers, like an amp url that is much, much cleaner to work with 

https://www.newsday.com/amp/opinion/letters/accessory-dwelling-unit-kathy-hochul-j37633

makes you wonder, what is that for? 

Sailthru, which is a personalization engine? 

There are JSON urls that seem to be for something called PageSuite: 
https://www.newsday.com/json/opinion/letters/accessory-dwelling-unit-kathy-hochul-j37633%22/

In [ ]:
soup.find(name="script", attrs={"type": "application/ld+json"})

<script type="application/ld+json">{"@context":"https://schema.org/","@type":"NewsArticle","mainEntityOfPage":"https://www.newsday.com/opinion/letters/accessory-dwelling-unit-kathy-hochul-j37633","headline":"Hochul's accessory apartment plan is hokum","image":"https://cdn.newsday.com/ace/c:ZjkwMzYzNmItNDNhYi00:OTQ0NGU1/landscape/1280","datePublished":"2022-02-13T10:00:00Z","dateModified":"2022-02-13T10:00:00Z","description":"Reader letters to Newsday.","articleSection":"Opinion","keywords":["Letters to the editor,Opinion"],"author":{"@type":"Person","name":"Newsday Readers","url":"https://www.newsday.com"},"publisher":{"@type":"Organization","name":"Newsday","logo":{"@type":"ImageObject","url":"https://cdn.newsday.com/projects/logos/nd-logo-square.jpg"}}}</script>

In [ ]:
eval(soup.head.script.text)['datePublished']

'2022-02-13T10:00:00Z'

In [ ]:
# how do we stop reading once we have the new h3 show up? 
# this is the way; break when we get to the h3. 
# we'll have to read it in several steps. 
letters = []

for sibling in soup.h3.next_siblings:
    if sibling.name == "h3":
        break
    else:
        for p in sibling.

Many families moved to the suburbs for their piece of America more than 50 years ago, when Long Island was country. Unfortunately, today our towns are inundated with illegal apartments, which are not taxed as such ["Hochul pulls housing plan," News, Feb. 18].
If everyone is so against making them legal, then why haven't our local governments enforced the laws? They turn a blind eye to multiple meters on houses and garages that house all these illegal apartments. How about landlords who rent out mattress space? This is missed revenue that puts a burden on homeowners, schools, fire departments, police, etc.
Why not make these landlords pay additional tax and take responsibility for their property? How about the absentee landlords who reap the rent money and don’t care about the neighborhood?

Those of us who live in towns have no defense and are ignored by local governments about upholding the law on these illegal dwellings.
Barbara Keilty Michaleski, West Hempstead 
Long Island’s afford

A funny thing that happens, and one of the reasons to beware of scraping in principle, is that websites change their layouts over time. At various times, the author's name has been formatted as: 

    <p><em>Alexander Janow, </em><em>Asharoken</em></p>
    <p><em>Thomas Tierney,</em></p><p>  <em>Greenlawn</em></p>
    <p><strong>— Glenn Tyranski, Huntington</strong></p>
    <p><strong>— Bruce Sander, Stony Brook</strong></p>
    <p><em>Andrew Grant,</em></p>
    <p><i>Michael Haroldsson, Wading River</i></p>

Which is unfortunate for our understanding of things. The <strong> tag appears to be the most current approach. The really nasty ones are the two paragraphs of text, where the author's name and hometown are put on different lines. Considerate but deadly. What we would want really is to detect the presence of a name, surrounded by some formatting. 

So many different formats for such critical information. Like I said, don't scrape unless you absolutely have to. 

we could also process all the articles and look for these things; the list of places in Long Island is not infinite. But, other people can write in too. 

always a name, sometimes a comma, sometimes a new line, sometimes 

In [ ]:
authors = [
    "<p><em>Alexander Janow, </em><em>Asharoken</em></p>",
    "<p><em>Thomas Tierney,</em></p><p><em>Greenlawn</em></p>",
    "<p><strong>— Glenn Tyranski, Huntington</strong></p>",
    "<p><strong>— Bruce Sander, Stony Brook</strong></p>",
    "<p><em>Andrew Grant,</em></p>",
    "<p><i>Michael Haroldsson, Wading River</i></p>"
    "<p>And now I want to talk about Jack McCoy. Law and Order and stuff.</p>"
]

In [ ]:
s = BeautifulSoup("<p><em>Thomas Tierney,</em></p><p><em>Greenlawn</em></p>")
for p in s.find_all('p'):
    print(p.text)
    all_p_text = "".join([e.get_text() for e in s.p.find_all("em")])
    print(all_p_text)
    print(p.text == "".join([e.get_text() for e in p.find_all("em")]) == p.text)

Thomas Tierney,
Thomas Tierney,
True
Greenlawn
Thomas Tierney,
True


This does a pretty decent job!

In [ ]:
for a in authors:
    s = BeautifulSoup(a)
    for p in s.find_all('p'):
        print(p.text)
        paragraph_text = "".join([e.get_text() for e in p.find_all(["em", "i", "strong"])]) 
        print(p.text == paragraph_text)

Alexander Janow, Asharoken
True
Thomas Tierney,
True
Greenlawn
True
— Glenn Tyranski, Huntington
True
— Bruce Sander, Stony Brook
True
Andrew Grant,
True
Michael Haroldsson, Wading River
True
And now I want to talk about Jack McCoy. Law and Order and stuff.
False


In [ ]:
nlp(BeautifulSoup("""<p><strong>— Bruce Sander, Stony Brook</strong></p>""").get_text()).ents[1].label_

'PERSON'

In [ ]:
BeautifulSoup("<p><em>Alexander Janow, </em><em>Asharoken</em></p>").get_text().replace(u'\xa0', ' ')

'Alexander Janow, Asharoken'

In [ ]:
BeautifulSoup("<p><em>Michael Haroldsson, Wading River</em></p>").find(["i", "em", "strong"])

<em>Michael Haroldsson, Wading River</em>

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
e = nlp("Dave O'Brien, Islip").ents[0]

In [ ]:
e.label_

'PERSON'

In [211]:
import json

def process_letters_to_editor_signature(html_file):

    with open(html_file) as fin:
        html_doc = fin.read()

    soup = BeautifulSoup(html_doc)

    # remove ads! required because sometimes they put ads in between the person's name and 
    # hometown. Yeesh. 
    for div in soup.find_all("div", attrs={"class": lambda x: x.startswith("ad") if x else False}):
        div.decompose()

    letter_content = soup.find(name='div', attrs={"class": "contentAccess"})

    articles = []
    topic = soup.main.article.header.h1.text
    metadata = soup.find(name="script", attrs={"type": "application/ld+json"})
    metadata_json = json.loads(metadata.get_text())
    amp_html = soup.find(name="link", attrs={"rel": "amphtml"})
    if amp_html:
        amp_url = amp_html.attrs["href"]
    else:
        amp_url = metadata_json["mainEntityOfPage"]
    published_date = metadata_json['datePublished']
    text = "" 
    author = ""
    found_author = False

    # look for a main topic outside of the content div
    contents = letter_content.find_all(["h1", "h2", "h3", "p"])

    for x in contents:
        # if we've processed the author, ignore what comes after
        if found_author:
            found_author = False
            continue

        # if h3, reset the topic
        if x.name in ["h1", "h2", "h3"]:
            topic = x.get_text()

        # only look at paragraphs
        # if you see a paragraph where every piece of text is formatted
        # that's likely a name... 
        if x.name == "p":
            if x.find(["em", "strong", "i"]):
                paragraph_text = "".join([e.get_text() for e in x.find_all(["em", "i", "strong"])]) 
                if paragraph_text == x.text:
                    author = x.text
                    # look for a next paragraph, if there is one
                    if x.next_sibling.name == "p":
                        if x.next_sibling.find(["em", "i", "strong"]):
                            sibling_text = "".join([e.get_text() for e in x.next_sibling.find_all(["em", "i", "strong"])]) 
                            author = author + " " + sibling_text
                            found_author = True
                    articles.append( {
                        "url": amp_url,
                        "topic": topic,
                        "author": author,
                        "text": text,
                        "date_published": published_date,
                    })
                    text = ""
                    author = ""
            else:
                if not x.find("em", "strong", "i"):
                    text = text + x.get_text() + '\n' 

    return pd.DataFrame(articles)

In [212]:
process_letters_to_editor_signature('/workspaces/long-island-digest/data/raw/newsday/letters/newsday-reader-letters-opinion-o62515.html')

,url,topic,author,text,date_published
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-o62515,"Criticism of Trump, awards show unfair","Judy Riccuiti, Bayville","A letter writer recently criticized President Donald Trump for not attending the Kennedy Center Honors award ceremony, writing that the artists deserved to be recognized by the president and first lady [“A presidential snub of Kennedy honors show,” Dec...",2020-01-05T10:00:13.294Z
1,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-o62515,"Criticism of Trump, awards show unfair","John van Acken, Bellerose",A presidential snub?\nWhy would President Donald Trump attend an awards show to honor a bunch of elitist stars who daily demean him? He’s too busy to give these elitists any recognition! \n,2020-01-05T10:00:13.294Z


In [213]:
html_files = glob("/workspaces/long-island-digest/data/raw/newsday/letters/*.html")

In [214]:
import random

In [215]:
more_articles = []
for h in random.choices(html_files, k=10):
    df = process_letters_to_editor_signature(h)
    more_articles.append(df)
pd.concat(more_articles)[['url', 'author', 'text']]

,url,author,text
0,https://www.newsday.com/amp/opinion/letters/newsday-opinion-reader-letters-l40052,"Al Lane, Yaphank","It is absolutely appalling to me that former Suffolk County District Attorney Thomas Spota, former Suffolk County Police Chief James Burke, and former Suffolk anti-corruption unit chief Christopher McPartland could be so corrupt for so many years witho..."
1,https://www.newsday.com/amp/opinion/letters/newsday-opinion-reader-letters-l40052,"Stuart Klein, Island Park","How can one not feel an overwhelming sense of decay, incompetence and corruption having infused itself into almost every aspect of Nassau County’s structure [""NIFA: Nassau surplus in ’20 not a long-term fix,"" News, April 30]? \nWe’ve allowed Nassau Cou..."
2,https://www.newsday.com/amp/opinion/letters/newsday-opinion-reader-letters-l40052,"Daniel Palermo, North Amityville","I believe that when President Joe Biden addressed Congress on Wednesday night and touched on systemic racism, some viewers could infer that most white people are racist ["" ‘America is rising anew,’ Biden declares,"" News, April 29]. I know I am not. \nI..."
3,https://www.newsday.com/amp/opinion/letters/newsday-opinion-reader-letters-l40052,"Bob Krauss, Plainview","A reader suggested that all private pensions be free from New York State tax like the government’s state pensions [""Why New Yorkers are fleeing south,"" Letters, April 30]. \nWhy don’t we have New York government pension recipients pay state taxes as pr..."
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-k20687,"Kristen Buckley, North Merrick","I am absolutely disgusted and angered to hear about what happened at the protest on Merrick Road on Tuesday [“New day of LI protests of Floyd death,” News, June 3].\nI can’t believe this could happen in my own neighborhood, in my own “community.” I tho..."
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-b42116,"Anita L. Hertell, Ronkonkoma","The Sept. 25 letter “‘Medicare for All’ not a ‘far left’ idea” quoted a Kaiser Family Foundation poll that said that 53% of Americans support Medicare for All.\nI have worked and paid into the program for 40 years, and will receive Medicare coverage be..."
0,https://www.newsday.com/amp/opinion/letters/letters-friction-between-donald-trump-and-intelligence-chiefs-e49646,"Richard Reif,Kew Gardens Hills","President Donald Trump doesn’t need a CIA intervention, as Sen. Chuck Schumer said. The CIA needs an intervention to correct its pitiful performance.\nThe CIA was wrong about Iraq’s weapons of mass destruction, blindsided by the rise of the Islamic Sta..."
1,https://www.newsday.com/amp/opinion/letters/letters-friction-between-donald-trump-and-intelligence-chiefs-e49646,"Hank Cierski,Port Jefferson Station","Director of National Intelligence Dan Coates, CIA Director Gina Haspel and FBI Director Christopher Wray testified to the Senate Intelligence Committee about their assessment of worldwide threats [“Intel: North Korea, not Mexico border, real threat to ..."
2,https://www.newsday.com/amp/opinion/letters/letters-friction-between-donald-trump-and-intelligence-chiefs-e49646,"John Weishahn,Islip",President Donald Trump said his intelligence experts were wrong. He should be compelled to disclose the source of his contrary intelligence. The intelligence community must document its sources. He should have to document his.\n
0,https://www.newsday.com/amp/opinion/letters/the-u-s-still-uses-plenty-of-fossil-fuel-i16878,"Roger Halvorsen, Garden City","Has Newsday noticed that virtually everyone on Long Island still drives a car powered by fossil fuels?\nBased on your Jan. 24 editorial, “Drill, baby, drill is wrongheaded,” expressing concern about plans to expand offshore oil drilling, apparently it ..."


In [ ]:
len(html_files)

1743

In [217]:
from tqdm import tqdm

In [219]:
most_articles = []
error_files = []
for h in tqdm(html_files):
    try:
        df = process_letters_to_editor_signature(h)
        most_articles.append(df)
    except AttributeError as e:
        print(f"Error found in {h}")
        error_files.append(h)
most_articles_df = pd.concat(most_articles)

 20%|█▉        | 344/1743 [00:13<00:58, 23.88it/s]

Error found in /workspaces/long-island-digest/data/raw/newsday/letters/newsday-letters-to-the-editor-2018-c32689.html


 97%|█████████▋| 1694/1743 [01:11<00:01, 25.96it/s]

Error found in /workspaces/long-island-digest/data/raw/newsday/letters/submitting-your-letter-nzj035rq.html


100%|██████████| 1743/1743 [01:13<00:00, 23.77it/s]


{"@context":"https://schema.org/","@type":"NewsArticle","mainEntityOfPage":"https://www.newsday.com/opinion/letters/newsday-letters-to-the-editor-2018-c32689","headline":"Newsday's letters to the editor of 2018","image":"https://cdn.newsday.com/ace/c:NDVkM2Y2ZDktZWMxNS00:ZGExZDAw/landscape/1280","datePublished":"2018-12-17T17:20:00.065Z","dateModified":"2018-12-17T17:20:00.065Z","description":"A sampling of views from our readers throughout the year.","articleSection":"Opinion","keywords":["Letters to the editor,Opinion,Letters to the Editor,NEWSDAY"],"author":{"@type":"Person","name":"The Editorial Board","url":"https://www.newsday.com"},"publisher":{"@type":"Organization","name":"Newsday","logo":{"@type":"ImageObject","url":"https://cdn.newsday.com/projects/logos/nd-logo-square.jpg"}}}
CSP Hash	
Attributes
No Attributes
Properties
#text "{"@context":"https://schema.org/","@type":"NewsArticle","mainEntityOfPage":"https://www.newsday.com/opinion/letters/newsday-letters-to-the-editor-2018-c32689","headline":"Newsday's letters to the editor of 2018","image":"https://cdn.newsday.com/ace/c:NDVkM2Y2ZDktZWMxNS00:ZGExZDAw/landscape/1280","datePublished":"2018-12-17T17:20:00.065Z","dateModified":"2018-12-17T17:20:00.065Z","description":"A sampling of views from our readers throughout the year.","articleSection":"Opinion","keywords":["Letters to the editor,Opinion,Letters to the Editor,NEWSDAY"],"author":{"@type":"Person","name":"The Editorial Board","url":"https://www.newsday.com"},"publisher":{"@type":"Organization","name":"Newsday","logo":{"@type":"ImageObject","url":"https://cdn.newsday.com/projects/logos/nd-logo-square.jpg"}}}"


In [221]:
most_articles_df.author.value_counts()

                                        18
Cynthia Lovecchio, Glen Cove            14
Richard M. Frauenglass, Huntington      14
Anthony Tanzi, Mastic Beach             13
Arthur M. Shatz, Oakland Gardens        13
                                        ..
Nancy Rosenthal and Barbara Epstein,     1
Hans Wenze, Farmingdale                  1
Leonard Tufaro, Medford                  1
Jennifer DeVito, Medford                 1
Bonnie Sigman, Woodmere                  1
Name: author, Length: 5332, dtype: int64

In [222]:
most_articles_df[most_articles_df.author.str.startswith('Cynthia Lovecchio')]

,url,author,text
6,https://www.newsday.com/amp/opinion/letters/letters-the-debate-over-the-federal-shutdown-and-the-wall-j33342,"Cynthia Lovecchio, Glen Cove","Donald Trump needs, among other things, a lesson in civics. He seems to believe Congress is subservient to the president, not an equal branch of government. He doesn’t appear to understand that he can’t achieve his goals by bullying or threatening its ..."
1,https://www.newsday.com/amp/opinion/letters/matt-whitaker-s-new-post-a-problem-o59421,"Cynthia Lovecchio, Glen Cove","While rightly concerned about Acting Attorney General Matt Whitaker’s bias on the Russia investigation, many people are missing a more important issue that should be bipartisan.\nHe worked for a company under investigation by the FBI, a law enforcement..."
3,https://www.newsday.com/amp/opinion/letters/newsday-letters-to-the-editor-for-friday-march-16-2018-h86226,"Cynthia Lovecchio, Glen Cove","It was disappointing to read that many of our local Republican and Conservative representatives accept support from the National Rifle Association through party accounts [“The indirect way NRA funds NY candidates,” News, March 1].\nWe need to hold loca..."
2,https://www.newsday.com/amp/opinion/letters/newsday-letters-to-the-editor-for-wednesday-june-13-2018-l73590,"Cynthia Lovecchio,Glen Cove","There is deep concern over the erosion of trust in the FBI, the rule of law and all law officers, due to President Donald Trump’s assaults on the judicial system. We don’t have to look further than the words of the jury foreman in the Manganos’ mistria..."
2,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-c00680,"Cynthia Lovecchio, Glen Cove","A reader defended President Donald Trump’s intrusion into the military justice system [“The president is in charge of the military,” Letters, Dec. 3].\nThe writer appears to share Trump’s disdain for honor, a chain of command and ethics. Our people in ..."
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-church-pope-francis-n30983,"Cynthia Lovecchio, Glen Cove","I am stunned by the Expressway column by a loyal Catholic who writes that she was devastated by the church’s failure to discipline a priest she knew to be an abuser [“Amid the scandals, I’m still Catholic,” March 3].\nI have one question: When you saw ..."
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-h04178,"Cynthia Lovecchio, Glen Cove","A reader criticized Northwell Health CEO Michael Dowling’s push to end gun violence as grandstanding [“Northwell CEO should fight medical errors,” Letters, Oct. 3]. \nBefore retirement, I worked for more than 20 years directly for Dowling or within his..."
2,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-i96870,"Cynthia Lovecchio, Glen Cove","I am not a strong fan of Gov. Andrew M. Cuomo, but I’ve come to admire his handling of the pandemic. I’ve read the complaints in recent letters and the statements of politicians critical of his actions regarding patient transfer and housing [""Cuomo sho..."
4,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-j86094,"Cynthia Lovecchio, Glen Cove","A recent letter epitomizes the lack of empathy underlying our current national behavior [“Forget ‘free’ tuition — I’ll pay for college,” July 20]. The writer wants everyone to vote on tax rates rather than plans and policies for the nation.\nThis “I go..."
0,https://www.newsday.com/amp/opinion/letters/newsday-reader-letters-opinion-m51328,"Cynthia Lovecchio, Glen Cove","Recent letters have equated Democrats with Republicans as sore losers [""Democrats are the sore losers,"" Letters, Nov. 30]. This is untrue. Hillary Clinton lost the 2016 election to Donald Trump by fewer votes in key states than Trump lost to Joe Biden...."
